# Mathematics

In [ ]:
import torch
import numpy as np
import scipy.stats as stats

import sympy as sym

import plotly.graph_objects as go
from plotly.io import renderers
renderers.default = 'png'
from plotly.subplots import make_subplots

from pprint import pprint
from utils.design import plotly_main_layout, plotly_main_subplot_layout

___
### Artificial Neural Network

> #### ANNs
> - $\hat{y}=\sigma(x_0 w_0 + \sum_{i=1}^m x_iw_i)$
> - $\hat{y}=\sigma(\textbf{x}^\text{T}\textbf{w}+w_0)$
> - $\hat{y}=\sigma(\textbf{x}^\text{T}\textbf{w})$
>   - $\sigma$: activation function
>   - $\textbf{x}$: input vector
>   - $\textbf{w}$: weights
>   - $w$: bias (intercept)

> #### Loss Function
> - $\mathcal{L}_\text{MSE}= \dfrac{1}{2}(\hat{y}-y)^2$
> - $\mathcal{L}_\text{Cross-Entropy}= -(y\log (\hat{y}) + (1-y)\log (1-\hat{y}))$

> #### Cost Function
> - $\mathcal{J} = \dfrac{1}{n}\sum_{i=1}^n \mathcal{L}(\hat{y}_i , y_i)$

___
### X

___
### X

___
### X

___
### X

___
### X

___
### X

___
### X